# EDA de **Telefonía Fija: Penetración (Totales)**

Cuaderno para **Visual Studio/VS Code** (Python) con pasos documentados en Markdown y exportación del CSV 'limpio'.

**Columnas esperadas:** `Registro_ID, Año, Trimestre, Accesos telefonía fija cada 100 hab, Accesos telefonía fija cada 100 hogares, TiempoTrim_ID`.

## 1. Importaciones y configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
> **Ruta proporcionada:**

`C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Penetración Totales_con_ID.csv`

Usa `encoding='utf-8-sig'` para preservar acentos/eñes en Windows.

In [ ]:
ruta = r'C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Telefonia Fija\\Telefonía Fija Penetración Totales_con_ID.csv'

df = pd.read_csv(ruta, encoding='utf-8-sig')
df_raw = df.copy()

print('Shape:', df.shape)
print('Columnas:', list(df.columns))
display(df.head())

## 3. Perfil inicial
- Tipos de datos (`df.info()`)
- Nulos y duplicados
- Estadísticos de penetración (hab/hogares)


In [ ]:
df.info()

print('
Nulos por columna:
', df.isna().sum())
print('Duplicados (filas completas):', df.duplicated().sum())

cols = ['Accesos telefonía fija cada 100 hab','Accesos telefonía fija cada 100 hogares']
for c in cols:
    if c in df.columns:
        print(c, 'min=', df[c].min(), 'max=', df[c].max(), 'mean=', float(df[c].mean()))

## 4. Validaciones y construcción de `Periodo`
- Conversión a numérico
- Chequeo de negativos
- Construcción de `Periodo` (YYYY-Qn)


In [ ]:
num_cols = ['Registro_ID','Año','Trimestre','Accesos telefonía fija cada 100 hab','Accesos telefonía fija cada 100 hogares','TiempoTrim_ID']
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print('
Negativos por columna:
', (df[num_cols] < 0).sum())

df['Periodo'] = pd.PeriodIndex.from_fields(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Limpieza y ordenamiento
- Eliminación de duplicados
- Orden por `Año`, `Trimestre`


In [ ]:
df = df.drop_duplicates().sort_values(['Año','Trimestre']).reset_index(drop=True)
display(df.head())

## 6. Variables derivadas
- QoQ/YoY y media móvil (4) para hab/hogares

In [ ]:
df['PenHab_QoQ_%'] = df['Accesos telefonía fija cada 100 hab'].pct_change() * 100
df['PenHab_YoY_%'] = df['Accesos telefonía fija cada 100 hab'].pct_change(4) * 100
df['PenHab_MovAvg4'] = df['Accesos telefonía fija cada 100 hab'].rolling(4).mean()
df['PenHog_QoQ_%'] = df['Accesos telefonía fija cada 100 hogares'].pct_change() * 100
df['PenHog_YoY_%'] = df['Accesos telefonía fija cada 100 hogares'].pct_change(4) * 100
df['PenHog_MovAvg4'] = df['Accesos telefonía fija cada 100 hogares'].rolling(4).mean()

display(df[['Año','Trimestre','Accesos telefonía fija cada 100 hab','PenHab_QoQ_%','PenHab_YoY_%','PenHab_MovAvg4','Accesos telefonía fija cada 100 hogares','PenHog_QoQ_%','PenHog_YoY_%','PenHog_MovAvg4']].tail(8))

## 7. Visualizaciones
- Series temporales con media móvil (hab/hogares)
- YoY y QoQ (hab/hogares)

In [ ]:
import matplotlib.ticker as mtick

# 7.1 Series temporales con media móvil
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Accesos telefonía fija cada 100 hab', marker='o', label='Hab/100', color='#2E86C1')
sns.lineplot(data=df.dropna(subset=['PenHab_MovAvg4']), x='Periodo', y='PenHab_MovAvg4', marker='o', label='MM4 Hab/100', color='#7D3C98')
plt.xticks(rotation=45, ha='right'); plt.title('Penetración por 100 hab (con MM4)'); plt.xlabel('Periodo'); plt.ylabel('Accesos/100 hab'); plt.legend(); plt.tight_layout(); plt.show()

plt.figure()
sns.lineplot(data=df, x='Periodo', y='Accesos telefonía fija cada 100 hogares', marker='o', label='Hog/100', color='#1ABC9C')
sns.lineplot(data=df.dropna(subset=['PenHog_MovAvg4']), x='Periodo', y='PenHog_MovAvg4', marker='o', label='MM4 Hog/100', color='#D35400')
plt.xticks(rotation=45, ha='right'); plt.title('Penetración por 100 hogares (con MM4)'); plt.xlabel('Periodo'); plt.ylabel('Accesos/100 hogares'); plt.legend(); plt.tight_layout(); plt.show()

# 7.2 YoY y QoQ (en %)
plt.figure()
sns.lineplot(data=df, x='Periodo', y='PenHab_YoY_%', marker='o', label='Hab YoY %', color='#16A085')
sns.lineplot(data=df, x='Periodo', y='PenHab_QoQ_%', marker='o', label='Hab QoQ %', color='#E67E22')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()); plt.xticks(rotation=45, ha='right'); plt.title('Variación de Penetración (Hab: YoY y QoQ)'); plt.xlabel('Periodo'); plt.ylabel('%'); plt.legend(); plt.tight_layout(); plt.show()

plt.figure()
sns.lineplot(data=df, x='Periodo', y='PenHog_YoY_%', marker='o', label='Hog YoY %', color='#2980B9')
sns.lineplot(data=df, x='Periodo', y='PenHog_QoQ_%', marker='o', label='Hog QoQ %', color='#8E44AD')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter()); plt.xticks(rotation=45, ha='right'); plt.title('Variación de Penetración (Hogares: YoY y QoQ)'); plt.xlabel('Periodo'); plt.ylabel('%'); plt.legend(); plt.tight_layout(); plt.show()

## 8. Guardado del archivo **limpio**
Se guarda el CSV en la **misma ruta** con la **leyenda `_limpio`** agregada al nombre, codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)